# Aula #26 – Processamento de Linguagem Natural & Análise de Sentimento

# Word2vec

Já vimos antes que é possível transformar um texto em _features_ numéricas. Uma sofisticação do método _Bag of words_ é incorporar o contexto das palavras vizinhas nessas _features_ (é comum chamar o vetor de _features_ numéricas de _embedding_).

Imagine que nossa janela de contexto (context window) tem tamanho 5 (2 palavras _antes_ e 2 palavras _depois_ da palavra _central_).

Então, se a frase fosse `The quick brown fox jumps over the lazy dog`, teríamos as seguintes janelas:

<img src="data/nb_figs/windows_word2vec.png" width="600"/>

Para cada uma das janelas formadas, temos o vetor correspondente a elas (usando o _Bag of words_ binário - com apenas 0s e 1s; também chamado de `one-hot encoding`):

<img src="data/nb_figs/one_hot_encoding_word2vec.png" width="600"/>

Há duas arquiteturas possíveis para se obter os `embeddings` word2vec. Uma delas é chamada de `CBoW` (_Continuous Bag of Words_) e outra é chamada de `Skip gram`. Aqui, vamos focar no `Skip gram`, que considera como input o vetor da palavra central da janela, e como output, os vetores do contexto. O objetivo do algoritmo é aprender os pesos da _hidden layer_, de forma que as probabilidades finais sejam condizentes com as co-ocorrências das palavras em nosso _corpus_ de documentos.

<img src="data/nb_figs/nn_word2vec_large.png" width="800"/>

Ao final do treinamento, a matriz correspondente à _hidden layer_, com 10 mil (tamanho do vocabulário) linhas e 300 (quantidade de dimensões do _embedding_) colunas será tal que cada linha representará o embedding de uma palavra do vocabulário.

Para saber mais sobre `word2vec`, leia em:

* http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
* https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/
* https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

## Similaridade entre ingredientes - uma aplicação do _word2vec_ a um dataset de receitas

O dataset utilizado é um dos datasets do site [Recipe box](https://eightportions.com/datasets/Recipes).

A ideia é treinar um modelo `word2vec` usando a biblioteca [gensim](https://radimrehurek.com/gensim/index.html) e depois construirmos uma aplicação pela qual seja possível obter uma lista dos ingredientes mais similares a um determinado ingrediente. Vamos tentar?

### Leitura do dataset

In [1]:
import pandas as pd

In [2]:
import json

In [3]:
with open('data/datasets/recipes/recipes_raw_nosource_ar.json') as f:
    recipes_list = list(json.load(f).values())

In [4]:
df = pd.DataFrame(recipes_list)

In [5]:
df = df[df['instructions'].str.len() > 0]

In [6]:
df.head()

,title,ingredients,instructions,picture_link
0,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S
1,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe
2,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56
3,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi
4,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm


In [7]:
len(df)

39522

In [8]:
import string
import nltk
import re

In [9]:
translation_table = str.maketrans({key: ' ' for key in string.punctuation}) 
def remove_punctuation(text):
    return text.translate(translation_table)

In [10]:
digits_regex = re.compile(r'[0-9]')
def remove_digits(text):
    return digits_regex.sub('', text)

In [11]:
stopwords = set(nltk.corpus.stopwords.words('english'))
def remove_stopwords(text):
    return [word for word in text.split() if word not in stopwords]

In [12]:
def text_to_normalized_tokens(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = remove_digits(text)
    text = remove_stopwords(text)
    return text

In [13]:
df['norm_instructions'] = df['instructions'].apply(text_to_normalized_tokens)

In [14]:
df = df[df['norm_instructions'].str.join('').str.len() > 0]

In [15]:
df.head()

,title,ingredients,instructions,picture_link,norm_instructions
0,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves AD...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S,"[place, chicken, butter, soup, onion, slow, co..."
1,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,"[slow, cooker, mix, cream, mushroom, soup, dry..."
2,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar ADVERTISEMENT, 1/2...",Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,"[preheat, oven, degrees, f, degrees, c, lightl..."
3,Best Chocolate Chip Cookies,"[1 cup butter, softened ADVERTISEMENT, 1 cup w...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,"[preheat, oven, degrees, f, degrees, c, cream,..."
4,Homemade Mac and Cheese Casserole,[8 ounces whole wheat rotini pasta ADVERTISEME...,Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm,"[preheat, oven, degrees, f, line, quart, casse..."


## Treinamento do _word2vec_

**Tarefa:** Treine um modelo word2vec usando os dados da coluna `ingredients` (`words_list`)

1. crie uma variável chamada `word_list` que é uma lista com os ingredientes

2. defina as variáveis `size` (tamanho do embedding) e `window` (tamanho da janela que deslizará pelas listas de palavras

3. faça o treinamento do word2vec, passando como parâmetros `word_list`, `size=size`, `window=window`, `min_count=1` e `workers=4` (vc pode aumentar esse número ou diminuir de acordo com a quantidade de cpus que vc tiver disponível)

Dica: Leia a documentação sobre a classe `Word2Vec`

<!-- 
words_list = df['norm_instruction'].tolist()
size = 300
window = 5
model = Word2Vec(words_list, size=size, window=window, min_count=1, workers=4)
-->

In [16]:
from gensim.models import Word2Vec

In [17]:
?Word2Vec

Init signature:
Word2Vec(
    sentences=None,
    corpus_file=None,
    size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    iter=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    max_final_vocab=None,
)
Docstring:     
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only querying)
store and use only the :class:`~gensim.models.keyedvectors.KeyedVectors` instance in `self.wv` to reduce memory.

The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save` and
:meth:`~gensim.models.word2vec.Word2Vec.load` methods.

The trained word vectors can also be stored/loaded from a format 

In [18]:
words_list = df['norm_instructions'].tolist()

In [19]:
print(words_list[:2])

[['place', 'chicken', 'butter', 'soup', 'onion', 'slow', 'cooker', 'fill', 'enough', 'water', 'cover', 'cover', 'cook', 'hours', 'high', 'minutes', 'serving', 'place', 'torn', 'biscuit', 'dough', 'slow', 'cooker', 'cook', 'dough', 'longer', 'raw', 'center'], ['slow', 'cooker', 'mix', 'cream', 'mushroom', 'soup', 'dry', 'onion', 'soup', 'mix', 'water', 'place', 'pot', 'roast', 'slow', 'cooker', 'coat', 'soup', 'mixture', 'cook', 'high', 'setting', 'hours', 'low', 'setting', 'hours']]


In [20]:
size = 300
window = 5

In [21]:
%%time
model = Word2Vec(words_list, size=size, window=window, workers=4)

CPU times: user 32.6 s, sys: 188 ms, total: 32.8 s
Wall time: 10 s


Se você quiser salvar o modelo, você precisa apenas utilizar o método `save`, passando como parâmetro o local em que deseja que o modelo seja salvo.

Por exemplo:
```python
model.save('data/gensim.model')
```

### Similaridade entre vetores

Em modelos vetoriais de linguagem, em geral, utiliza-se a similaridade de cosseno como medida de similaridade entre dois vetores, já que ela captura a noção de que vetores apontando para a mesma direção são próximos.

In [22]:
from scipy.spatial.distance import cosine

In [23]:
lime_vec = model.wv['lime']
lemon_vec = model.wv['lemon']

In [24]:
def similarity_between_vec(vec1, vec2):
    return 1 - cosine(vec1, vec2)

In [25]:
similarity_between_vec(lime_vec, lemon_vec)

0.811663031578064

### Termos mais comuns

Vamos ver quais são os termos mais comuns do dataset?

In [26]:
from collections import Counter

In [27]:
from itertools import chain

In [28]:
all_words = chain.from_iterable(df['norm_instructions'].tolist())

In [29]:
Counter(all_words).most_common(20)

[('minutes', 60807),
 ('heat', 44501),
 ('stir', 40642),
 ('degrees', 40360),
 ('bowl', 32988),
 ('oven', 32953),
 ('mixture', 32805),
 ('cook', 28047),
 ('medium', 25904),
 ('add', 25235),
 ('large', 25079),
 ('place', 23035),
 ('salt', 22842),
 ('mix', 22016),
 ('sugar', 21408),
 ('baking', 21166),
 ('water', 21160),
 ('f', 20765),
 ('pepper', 20350),
 ('c', 19586)]

### Os mais próximos

Um método legal do objeto `Word2VecKeyedVectors` é o `most_similar`, que retorna as palavras mais similares a uma determinada palavra. Note que podemos modificar a quantidade de itens retornados, colocando um valor para parâmetro `topn` (por padrão, ele é 10).

**Tarefa:** brinque até ficar satisfeito.

As relações fazem sentido?

In [30]:
?model.wv.most_similar

Signature:
model.wv.most_similar(
    positive=None,
    negative=None,
    topn=10,
    restrict_vocab=None,
    indexer=None,
)
Docstring:
Find the top-N most similar words.
Positive words contribute positively towards the similarity, negative words negatively.

This method computes cosine similarity between a simple mean of the projection
weight vectors of the given words and the vectors for each word in the model.
The method corresponds to the `word-analogy` and `distance` scripts in the original
word2vec implementation.

Parameters
----------
positive : list of str, optional
    List of words that contribute positively.
negative : list of str, optional
    List of words that contribute negatively.
topn : int or None, optional
    Number of top-N similar words to return, when `topn` is int. When `topn` is None,
    then similarities for all words are returned.
restrict_vocab : int, optional
    Optional integer which limits the range of vectors which
    are searched for most-simil

In [31]:
model.wv.most_similar('rice')

[('quinoa', 0.6486576795578003),
 ('lentils', 0.6182992458343506),
 ('barley', 0.5376075506210327),
 ('couscous', 0.49438077211380005),
 ('farro', 0.49112915992736816),
 ('millet', 0.486141562461853),
 ('absorbed', 0.48095762729644775),
 ('noodles', 0.4803483486175537),
 ('vegetables', 0.4523196220397949),
 ('saffron', 0.4370798170566559)]

## Visualização das relações entre os ingredientes

Vamos agora construir funções que permitem:

1. buscar o nome de um ingrediente
2. retornar os termos mais próximos (que não são ele mesmo)

In [32]:
VOCAB = set(model.wv.vocab.keys())

**Tarefa:** Complete a função abaixo, que dado um termo (`word`), retorna os `n` termos mais similares. Além disso, se a palavra não está no vocabulário, imprime uma mensagem que avisa o usuário que a palavra não existe no vocabulário.

Lembre-se de passar o parâmetro `topn` (`topn=n`) para o método `model.wv.most_similar` para retornar os `n` termos.

<!-- 
def get_similar(word, n):
    if word not in VOCAB:
        print(f'A palavra "{word}" não está em nosso vocabulário!')
        exit(1)
    
    most_similar_words = []
    for similar_word, distance in model.wv.most_similar(word, n=n+1):
        most_similar_words.append(similar_word)
        
    return most_similar_words
-->

In [33]:
def get_similar(word, n=10):
    if word not in VOCAB:
        print(f'A palavra "{word}" não está em nosso vocabulário!')
        exit(1)
    
    most_similar_words = []
    for similar_word, distance in model.wv.most_similar(word, topn=n+1):
        most_similar_words.append(similar_word)
        
    return most_similar_words

In [34]:
def search_text(text, max_display=15):
    words_with_text = [word for word in VOCAB if text.lower() in word][:max_display]
    if text in words_with_text:
        words_with_text = [text] + [word for word in words_with_text if text != word]
    return words_with_text

In [35]:
search_text('rosemary')

['rosemary']

In [36]:
get_similar('rosemary')

['thyme',
 'sage',
 'marjoram',
 'tarragon',
 'oregano',
 'sprigs',
 'basil',
 'parsley',
 'savory',
 'paprika',
 'dill']

**Tarefa bônus:** descubra a similaridade entre uma receita e outra receita através dos embeddings que treinamos acima. Isso envolve utilizar um embedding para a instrução (a nossa "sentença", que é um conjunto de palavras).

Veja exemplos da criação de embeddings para sentença a partir dos embeddings da palavra [neste post](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/).

Após a criação dos embeddings das receitas, basta comparará-las!